In [1]:
import gpytorch
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torchvision import transforms

gpytorch.functions.use_toeplitz = False

In [2]:
class FeatureExtractor(nn.Sequential):
    
    def __init__(self):
        super(FeatureExtractor, self).__init__(nn.Conv2d(1, 32, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(32),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2),
                                 nn.Conv2d(32, 64, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2))
        
class Bottleneck(nn.Sequential):
    
    def __init__(self):
        super(Bottleneck, self).__init__(nn.Linear(64*7*7, 128),
                                         nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128, 128),
                                 nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128,64),
                                 nn.BatchNorm1d(64))

class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.bottleneck = Bottleneck()
        self.final_layer = nn.Sequential(
                                 nn.ReLU(),
                                 nn.Linear(64,10))
    
    def forward(self, x):
        features = self.feature_extractor(x)
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        classification = self.final_layer(bottlenecked_features)
        return classification
        

In [3]:
train_mnist = torchvision.datasets.MNIST('/mnt/bigboi/datasets/mnist/', train=True,
                                         download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))                                              
test_mnist = torchvision.datasets.MNIST('/mnt/bigboi/datasets/mnist/', train=False,
                                        download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

In [4]:
train_data_loader = torch.utils.data.DataLoader(train_mnist, shuffle=True, pin_memory=True, batch_size=256)

In [5]:
criterion = nn.CrossEntropyLoss().cuda()

In [6]:
model = LeNet().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [7]:
num_epochs = 0
if num_epochs > 0:
    for i in range(num_epochs):
        for x, y in train_data_loader:
            optimizer.zero_grad()
            x = Variable(x.cuda())
            y = Variable(y.cuda())
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
        print("Loss: %.3f" % loss.data[0])
    torch.save(model.state_dict(), '/mnt/data/dkl/mnist/lenet.dat')
else:
    model.load_state_dict(torch.load('/mnt/data/dkl/mnist/lenet.dat'))
    
model.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)
avg = 0.
i = 0.
for test_batch_x, test_batch_y in test_data_loader:
    predictions = model(Variable(test_batch_x).cuda()).max(-1)[1]
    test_batch_y = Variable(test_batch_y).cuda()
    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
    i += 1.
print('Accuracy: %.4f' % (avg / i))

Accuracy: 0.9928


In [8]:
list(model.bottleneck.modules())[-1].weight.data.fill_(1)
None

In [9]:
from gpytorch.kernels import RBFKernel, GridInterpolationKernel

class DeepKernel(gpytorch.Module):
    def __init__(self, model):
        super(DeepKernel, self).__init__()
        self.feature_extractor = model.feature_extractor
        self.bottleneck = model.bottleneck
        self.gp_layer = GPLayer()
        
    def forward(self, x):
        features = self.feature_extractor(x)
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        gp_output = self.gp_layer(bottlenecked_features)
        return gp_output
    
    
class LatentFunction(gpytorch.AdditiveGridInducingPointModule):
    def __init__(self):
        super(LatentFunction, self).__init__(grid_size=256, grid_bounds=[(-7, 7)],
                                             n_components=64, mixing_params=False, sum_output=False)
        cov_module = RBFKernel()
        cov_module.initialize(log_lengthscale=2)
        self.cov_module = cov_module
        
    def forward(self, x):
        mean = Variable(x.data.new(len(x)).zero_())
        covar = self.cov_module(x)
        return gpytorch.random_variables.GaussianRandomVariable(mean, covar)

    
class GPLayer(gpytorch.GPModel):
    def __init__(self, n_dims=64):
        super(GPLayer, self).__init__(gpytorch.likelihoods.SoftmaxGaussianLikelihood(n_features=64, n_classes=10, rank=5))
        self.latent_function = LatentFunction()
    
    def forward(self, x):
        res = self.latent_function(x)
        return res
    

In [10]:
len(train_mnist)

60000

In [11]:
deep_kernel = DeepKernel(model).cuda()
gp_data_loader = torch.utils.data.DataLoader(train_mnist, batch_size=2048., pin_memory=True, shuffle=True)

In [14]:
# Find optimal model hyperparameters
deep_kernel.train()
optimizer = torch.optim.Adam(deep_kernel.gp_layer.parameters(), lr=0.01)
optimizer.n_iter = 0
for i in range(10):
    for j, (train_x_batch, train_y_batch) in enumerate(gp_data_loader):
        train_x_batch = Variable(train_x_batch).cuda()
        train_y_batch = Variable(train_y_batch).cuda()
        optimizer.zero_grad()
        output = deep_kernel(train_x_batch)
        loss = -deep_kernel.gp_layer.marginal_log_likelihood(output, train_y_batch, n_data=len(train_mnist))
        kl = deep_kernel.gp_layer.likelihood.kl_div() / len(train_mnist)
        loss = loss + kl
        loss.backward()
        optimizer.n_iter += 1
        print('Iter %d/200 - Loss: %.3f' % (
            i + 1, loss.data[0],
        ))
        optimizer.step()
        
    deep_kernel.eval()
    test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

    avg = 0.
    i = 0.
    for test_batch_x, test_batch_y in test_data_loader:
        predictions = deep_kernel(Variable(test_batch_x).cuda()).argmax()
        test_batch_y = Variable(test_batch_y).cuda()
        avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
        i += 1.

    print('Score')
    print(avg / i)
    deep_kernel.train()


Iter 1/200 - Loss: -0.193
Iter 1/200 - Loss: -0.197
Iter 1/200 - Loss: -0.220
Iter 1/200 - Loss: 0.112
Iter 1/200 - Loss: -0.307
Iter 1/200 - Loss: -0.283
Iter 1/200 - Loss: -0.297
Iter 1/200 - Loss: -0.271
Iter 1/200 - Loss: -0.278
Iter 1/200 - Loss: -0.265
Iter 1/200 - Loss: -0.272
Iter 1/200 - Loss: -0.266
Iter 1/200 - Loss: -0.178
Iter 1/200 - Loss: -0.278
Iter 1/200 - Loss: -0.319
Iter 1/200 - Loss: -0.289
Iter 1/200 - Loss: -0.276
Iter 1/200 - Loss: -0.246
Iter 1/200 - Loss: -0.206
Iter 1/200 - Loss: -0.220
Iter 1/200 - Loss: -0.227
Iter 1/200 - Loss: -0.275
Iter 1/200 - Loss: -0.295
Iter 1/200 - Loss: -0.252
Iter 1/200 - Loss: -0.226
Iter 1/200 - Loss: -0.135
Iter 1/200 - Loss: -0.226
Iter 1/200 - Loss: -0.231
Iter 1/200 - Loss: -0.246
Iter 1/200 - Loss: -0.221
Score
0.99013671875
Iter 2/200 - Loss: -0.217
Iter 2/200 - Loss: -0.175
Iter 2/200 - Loss: -0.171
Iter 2/200 - Loss: -0.204
Iter 2/200 - Loss: -0.219
Iter 2/200 - Loss: -0.186
Iter 2/200 - Loss: -0.198
Iter 2/200 - Loss: 

KeyboardInterrupt: 

In [13]:
deep_kernel.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

avg = 0.
i = 0.
for test_batch_x, test_batch_y in test_data_loader:
    print(deep_kernel(Variable(test_batch_x).cuda()))
    predictions = deep_kernel(Variable(test_batch_x).cuda()).argmax()
    test_batch_y = Variable(test_batch_y).cuda().float()
    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
    i += 1.

print(avg / i)


Variable containing:
 1.9288e-05  3.3848e-17  8.6510e-08  ...   9.9997e-01  1.8086e-08  5.9959e-06
 6.9664e-07  1.9114e-05  9.7293e-01  ...   1.0703e-06  3.4917e-13  4.8554e-07
 3.8917e-15  1.0000e+00  3.7779e-09  ...   1.2948e-16  3.3263e-07  1.4373e-09
                ...                   ⋱                   ...                
 3.6423e-11  3.7937e-08  1.5453e-07  ...   7.8897e-07  4.8839e-08  8.0079e-12
 1.5432e-04  1.5667e-14  4.6644e-07  ...   9.9983e-01  7.7406e-07  1.2132e-05
 6.3759e-05  5.4233e-13  1.6788e-07  ...   9.9982e-01  1.6971e-05  9.7698e-05
[torch.cuda.FloatTensor of size 256x10 (GPU 0)]



RuntimeError: Expected object of type Variable[torch.cuda.LongTensor] but found type Variable[torch.cuda.FloatTensor] for argument #1 'other'